In [1]:
import numpy as np
import tensorflow 
import keras
import matplotlib.pyplot as plt

In [2]:
def define_discriminator(in_shape = (28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid', input_shape = in_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.MaxPool2D((3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [3]:
def define_generator(noise):
    gen_model = keras.Sequential()
    gen_model.add(keras.layers.Dense(150, input_dim = noise))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(250))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(450))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(600))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(700))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(784, activation = 'sigmoid'))
    gen_model.add(keras.layers.Reshape((28, 28, 1)))
#     gen_model.add(keras.activations.sigmoid())
    gen_model.add(keras.layers.Dropout(0.3))
    return gen_model

In [4]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = keras.Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [5]:
def load_real_samples():
    (train_X, _), (_, _) = keras.datasets.mnist.load_data()
    train_X = np.expand_dims(train_X, axis = -1)
    train_X = train_X/255
    return train_X

In [6]:
def generate_real_samples(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    Y = np.ones((n_samples, 1))
    return X, Y

In [7]:
def generate_latent_points(latent_dim, n_samples):
    noise = np.random.randn(latent_dim * n_samples)
    noise = noise.reshape(n_samples, latent_dim)
    return noise

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    g_input = generate_latent_points(latent_dim, n_samples)
    fake_data = g_model.predict(g_input)
    Y = np.zeros((n_samples, 1))
    return fake_data, Y

In [9]:
def visualise(examples, epoch, n = 10):
    for i in range(n * n):
        plt.subplot(n, n, i + 1)
        plt.imshow(examples[i, :, :, :], cmap = 'gray')
    file_name = f'C:/Users/bhatt/OneDrive/Desktop/whisper-main/DS_PP_Classroom_14_Oct/Machine Learning/{epoch}_EPOCH.jpg'
    plt.savefig(file_name)

In [10]:
def evaluate(g_model, d_model, dataset, latent_dim, epoch, batch = 100):
    X_real, Y_real = generate_real_samples(dataset, batch)
    
    _, real_acc = d_model.evaluate(X_real, Y_real)
    
    X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch)
    
    _, fake_acc = d_model.evaluate(X_fake, Y_fake)
    
    print(f'Accuracy real: {real_acc}, Accuracy fake: {fake_acc}')
    
    visualise(X_fake, epoch)

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, epoch, batch_size = 100):
    batch_per_epoch = int(dataset.shape[0]/batch_size)
    half_batch = int(batch_size/2)
    
    for i in range(epoch):
        for j in range(batch_per_epoch):
            X_real, Y_real = generate_real_samples(dataset, batch_size)
            
            X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch_size)
            
            d_loss_real, _ = d_model.train_on_batch(X_real, Y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, Y_fake)
            
            total_d_loss = d_loss_fake + d_loss_real
            
            X_noise = generate_latent_points(latent_dim, batch_size)
            
            Y_g_model = np.ones((batch_size, 1))
            
            g_loss, _ = gan_model.train_on_batch(X_noise, Y_g_model)
            
            print(f'D_loss: {total_d_loss}, G_loss: {g_loss}')
        
        if (i % 10 == 0):
            evaluate(g_model, d_model, dataset, latent_dim, i)

In [ ]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim, 100)